In [1]:
%pip install instaloader


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import requests
import uuid # 고유 ID 생성을 위해 추가
from datetime import datetime # 타임스탬프 생성을 위해 추가

# --- 설정 ---
INSTA_ID = "본인의_아이디" # 아이디 입력
HASHTAGS = ["영화의전당", "영화의전당맛집"] 
TARGET_COUNT = 50 # 태그당 수집할 개수

# 1. 저장 메인 폴더 생성
if not os.path.exists("image"):
    os.makedirs("image")

# 2. 브라우저 실행
options = webdriver.ChromeOptions()
# options.add_argument("--headless") # 차단 방지를 위해 가급적 비활성 유지
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 3. 로그인 단계
driver.get("https://www.instagram.com/")
print("🚀 로그인 창이 뜨면 자동으로 아이디를 입력합니다.")

try:
    time.sleep(3) 
    id_input = driver.find_element(By.NAME, "username")
    id_input.send_keys(INSTA_ID)
    print("✅ 아이디 입력 완료. 비밀번호를 입력하고 로그인을 마무리하세요.")
except Exception as e:
    print(f"자동 입력 중 오류(수동 로그인 진행): {e}")

print("\n" + "="*50)
print("⚠️  [필수] 로그인을 완료하세요.")
print("⚠️  메인 화면이 보이면 여기(터미널)서 엔터(Enter)를 치세요.")
print("="*50)
input("waiting...") 

# 4. 해시태그 수집 반복문
for tag in HASHTAGS:
    print(f"\n🔍 '{tag}' 수집 시작...")
    driver.get(f"https://www.instagram.com/explore/tags/{tag}/")
    time.sleep(5) 

    # 태그별 개별 폴더 생성
    tag_path = f"image/{tag}"
    if not os.path.exists(tag_path):
        os.makedirs(tag_path)

    collected_count = 0
    visited_urls = set()

    while collected_count < TARGET_COUNT:
        # 현재 화면의 이미지 요소들 가져오기
        images = driver.find_elements(By.TAG_NAME, "img")
        
        for img in images:
            try:
                img_url = img.get_attribute("src")
                
                # 기본 필터링 (아이콘, 중복, 유효하지 않은 주소 제외)
                if not img_url or "static" in img_url or img_url in visited_urls:
                    continue

                visited_urls.add(img_url)
                
                # --- [파일명 고유화 로직] ---
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                unique_id = uuid.uuid4().hex[:8] 
                filename_only = f"{tag}_{timestamp}_{unique_id}.jpg"
                save_full_path = os.path.join(tag_path, filename_only)
                # --------------------------

                # 이미지 다운로드
                response = requests.get(img_url, timeout=10)
                if response.status_code == 200:
                    with open(save_full_path, "wb") as f:
                        f.write(response.content)
                    
                    collected_count += 1
                    if collected_count % 10 == 0: 
                        print(f"   [进度] {collected_count}/{TARGET_COUNT} 완료")
                    
                    if collected_count >= TARGET_COUNT:
                        break
            except Exception:
                continue

        # 5. 스크롤 및 로딩 대기 (효율화 핵심)
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2.5) # 인스타그램 서버에 무리를 주지 않으면서 로딩 기다리기

    print(f"✅ '{tag}' 수집 완료 ({collected_count}장 저장됨)")

print("\n🎉 모든 수집이 종료되었습니다.")
driver.quit()

🚀 로그인 창이 뜨면 자동으로 아이디를 입력합니다.
자동 입력 중 오류(수동 로그인 진행): Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="username"]"}
  (Session info: chrome=144.0.7559.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x387d13
	0x387d54
	0x1ab290
	0x1f56ea
	0x1f598b
	0x237912
	0x218004
	0x235111
	0x217d56
	0x1e94d9
	0x1ea294
	0x5fbb64
	0x5f7215
	0x613fad
	0x3a1ef8
	0x3a9b0d
	0x390738
	0x390902
	0x37a1da
	0x76df5d49
	0x77d8d5db
	0x77d8d561
	(nil)


⚠️  [필수] 로그인을 완료하세요.
⚠️  로그인 후 '정보 저장' 팝업이 뜨면 '나중에 하기'를 누르세요.
⚠️  메인 피드 화면이 보이면 여기(터미널)서 엔터(Enter)를 치세요.

🔍 '영화의전당' 수집 시작...
   [10/100] 완료
   [20/100] 완료
   [30/100] 완료
   [40/100] 완료
   [50/100] 완료
   [60/100] 완료
   [70/100] 완료
   [80/100] 완료
   [90/100] 완료
   [100/100] 완료

🔍 '영화의전당맛집' 수집 시작...
   [10/100] 완료
   [20/100] 완료
   [30/100] 완료
   [40/100] 완료

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import requests
import uuid
from datetime import datetime
from PIL import Image
from io import BytesIO

# --- 1. 설정 및 태그 확장 ---
INSTA_ID = "ibo12224@naver.com"
# 수집하고 싶은 핵심 장소 리스트
base_locations = ["영화의전당","영화의전당맛집"]

# 외형적 특징을 이끌어낼 접미사
suffixes = [
    "건축물", "외관", "전경", "야경", "풍경", 
    "파사드", "조형물", "광장", "드론샷", "스카이라인"
]
# --- 포스터 필터링 강화 로직 (이미지 분석 텍스트 활용) ---
EXCLUDE_KEYWORDS = ["포스터", "티켓", "예매", "공연", "상영", "이벤트", "안내문", "광고"]

TARGET_COUNT_PER_TAG = 20 # 태그당 수집 개수 (태그가 많으므로 적절히 조절)
MIN_WIDTH, MIN_HEIGHT = 350, 350 # 썸네일 방지 최소 해상도

# --- 2. 브라우저 실행 및 로그인 ---
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.instagram.com/")
print("🚀 로그인 후 메인 피드가 나오면 터미널에서 엔터를 치세요.")
input("Waiting for login...")

# --- 3. 확장 태그 수집 루프 ---
for location in base_locations:
    # 해당 장소의 메인 폴더 생성 (예: image/영화의전당)
    main_save_path = f"image/{location}"
    if not os.path.exists(main_save_path):
        os.makedirs(main_save_path)

    # 확장 태그 리스트 생성 (기본 장소명 + 특징 접미사들)
    combined_tags = [location] + [f"{location}{s}" for s in suffixes]
    
    for tag in combined_tags:
        print(f"\n🔍 태그 수집 중: #{tag}")
        driver.get(f"https://www.instagram.com/explore/tags/{tag}/")
        time.sleep(5)

        collected_count = 0
        visited_urls = set()
        fail_count = 0

        while collected_count < TARGET_COUNT_PER_TAG:
            images = driver.find_elements(By.TAG_NAME, "img")
            saved_in_this_scroll = 0

            for img in images:
                try:
                    alt_content = img.get_attribute("alt")

                    # 1. 포스터 관련 키워드가 alt 텍스트에 있으면 제외
                    if any(key in alt_content for key in EXCLUDE_KEYWORDS):
                        continue

                    img_url = img.get_attribute("src")
                    if not img_url or "static" in img_url or img_url in visited_urls:
                        continue
                    
                    visited_urls.add(img_url)

                    # 이미지 규격 필터링 (썸네일/아이콘 제거)
                    res = requests.get(img_url, timeout=10)
                    img_data = Image.open(BytesIO(res.content))
                    w, h = img_data.size
                    
                    if w < MIN_WIDTH or h < MIN_HEIGHT:
                        continue

                    # 파일명 고유화 (장소명_시간_고유ID)
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    uid = uuid.uuid4().hex[:6]
                    filename = f"{location}_{timestamp}_{uid}.jpg"
                    
                    # 저장 (태그별이 아닌 '장소별' 통합 폴더에 저장하여 데이터셋 관리 용이)
                    with open(os.path.join(main_save_path, filename), "wb") as f:
                        f.write(res.content)

                    collected_count += 1
                    saved_in_this_scroll += 1
                    if collected_count >= TARGET_COUNT_PER_TAG: break

                except:
                    continue

            # 스크롤 및 탈출 조건
            if saved_in_this_scroll == 0:
                fail_count += 1
                if fail_count > 5: break
            else:
                fail_count = 0
            
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(3)

        print(f"✅ {tag} 완료 ({collected_count}장)")

driver.quit()
print("\n🎉 모든 장소의 특징점 데이터 수집이 완료되었습니다.")

🚀 로그인 후 메인 피드가 나오면 터미널에서 엔터를 치세요.

🔍 태그 수집 중: #영화의전당
✅ 영화의전당 완료 (20장)

🔍 태그 수집 중: #영화의전당건축물
✅ 영화의전당건축물 완료 (0장)

🔍 태그 수집 중: #영화의전당외관
✅ 영화의전당외관 완료 (0장)

🔍 태그 수집 중: #영화의전당전경
✅ 영화의전당전경 완료 (0장)

🔍 태그 수집 중: #영화의전당야경
✅ 영화의전당야경 완료 (18장)

🔍 태그 수집 중: #영화의전당풍경
✅ 영화의전당풍경 완료 (0장)

🔍 태그 수집 중: #영화의전당파사드
✅ 영화의전당파사드 완료 (0장)

🔍 태그 수집 중: #영화의전당조형물
✅ 영화의전당조형물 완료 (0장)

🔍 태그 수집 중: #영화의전당광장
✅ 영화의전당광장 완료 (3장)

🔍 태그 수집 중: #영화의전당드론샷
✅ 영화의전당드론샷 완료 (0장)

🔍 태그 수집 중: #영화의전당스카이라인
✅ 영화의전당스카이라인 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집
✅ 영화의전당맛집 완료 (3장)

🔍 태그 수집 중: #영화의전당맛집건축물
✅ 영화의전당맛집건축물 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집외관
✅ 영화의전당맛집외관 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집전경
✅ 영화의전당맛집전경 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집야경
✅ 영화의전당맛집야경 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집풍경
✅ 영화의전당맛집풍경 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집파사드
✅ 영화의전당맛집파사드 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집조형물
✅ 영화의전당맛집조형물 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집광장
✅ 영화의전당맛집광장 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집드론샷
✅ 영화의전당맛집드론샷 완료 (0장)

🔍 태그 수집 중: #영화의전당맛집스카이라인
✅ 영화의전당맛집스카이라인 완료 (0장)

🎉 모든 장소의 특징점 데이터 수집이 완료되었습니다.


In [7]:
%pip install imagehash

   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   ---------------------------------------- 4.2/4.2 MB 35.9 MB/s  0:00:00

   ---------------------------------------- 0/2 [PyWavelets]
   ---------------------------------------- 0/2 [PyWavelets]
   ---------------------------------------- 0/2 [PyWavelets]
   ---------------------------------------- 0/2 [PyWavelets]
   ---------------------------------------- 0/2 [PyWavelets]
   ---------------------------------------- 0/2 [PyWavelets]
   ---------------------------------------- 0/2 [PyWavelets]
   ---------------------------------------- 0/2 [PyWavelets]
   -------------------- ------------------- 1/2 [imagehash]
   ---------------------------------------- 2/2 [imagehash]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time, os, requests, uuid
from datetime import datetime
from PIL import Image
from io import BytesIO
import imagehash

# --- 1. 설정: 장소 리스트 및 필터링 ---
locations = [
    "영화의전당", "호천마을", "국제시장", "화명생태공원", "동래읍성", 
    "벡스코", "전포카페거리", "자갈치시장", "문화빙상센터", "동래온천", 
    "더베이101", "부전마켓타운", "비프광장", "구포시장", "금강공원", 
    "누리마루", "송상현광장", "용두산공원", "부산어촌민속관", 
    "해운대해수욕장", "황령산", "보수동책방골목", "해운대블루라인파크", 
    "성지곡수원지", "장산", "송정해수욕장"
]
base_locations = sorted(locations)

tag_variants = ["", "뷰", "포토존", "출사", "나들이", "여행", "야경", "광장"]

# 포스터 및 텍스트 차단 키워드
EXCLUDE_ALT = [
    "텍스트", "포스터", "글자", "문구", "티켓", "안내문", 
    "이벤트", "상영표", "공연", "광고", "현수막", "스크린샷", "전단지"
]

MAX_SCROLL_PER_TAG = 5   # 태그당 최대 스크롤
TARGET_COUNT_TOTAL = 100 # 장소당 목표 개수
image_fingerprints = set()
visited_urls = set()

# --- 2. 브라우저 설정 ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.instagram.com/")
input("🚀 로그인을 완료하고 '메인 피드'가 뜨면 엔터를 치세요...")

# --- 3. 수집 루프 ---
for location in base_locations:
    save_path = f"image/{location}"
    if not os.path.exists(save_path): os.makedirs(save_path)
    
    collected_in_location = 0
    print(f"\n📍 [{location}] 수집 시작")

    for var in tag_variants:
        if collected_in_location >= TARGET_COUNT_TOTAL: break
        
        target_tag = f"{location}{var}"
        print(f"  🔍 #{target_tag}", end=" ", flush=True)
        driver.get(f"https://www.instagram.com/explore/tags/{target_tag}/")

        # [조치 1] 지능형 로딩 대기 (최대 15초)
        try:
            wait = WebDriverWait(driver, 15)
            # 첫 번째 이미지가 뜰 때까지 대기
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "img")))
            
            # [조치 2] 결과 없음 페이지 즉시 감지
            no_results = driver.find_elements(By.XPATH, "//span[contains(text(), '결과를 찾을 수 없음')]")
            if no_results:
                print("-> ❌ 결과 없음 (Skip)")
                continue
        except:
            print("-> ⚠️ 로딩 실패/결과 없음 (Skip)")
            continue

        print("-> 📥 수집중...", end="")

        for scroll in range(MAX_SCROLL_PER_TAG):
            time.sleep(2) # 렌더링 시간 확보
            img_elements = driver.find_elements(By.TAG_NAME, "img")
            saved_before_scroll = collected_in_location
            
            for img in img_elements:
                try:
                    img_url = img.get_attribute("src")
                    alt_text = img.get_attribute("alt") or ""
                    
                    if not img_url or img_url in visited_urls or "static" in img_url: continue
                    if any(ex in alt_text for ex in EXCLUDE_ALT): continue

                    res = requests.get(img_url, timeout=5)
                    img_obj = Image.open(BytesIO(res.content))
                    
                    # 지문(Hash) 비교
                    curr_hash = str(imagehash.average_hash(img_obj))
                    if curr_hash in image_fingerprints:
                        visited_urls.add(img_url)
                        continue
                    
                    w, h = img_obj.size
                    if w < 400 or h < 400: continue

                    # 저장
                    image_fingerprints.add(curr_hash)
                    visited_urls.add(img_url)
                    filename = f"{location}_{uuid.uuid4().hex[:6]}.jpg"
                    img_obj.convert("RGB").save(os.path.join(save_path, filename), "JPEG")

                    collected_in_location += 1
                    if collected_in_location >= TARGET_COUNT_TOTAL: break
                except: continue
            
            if collected_in_location >= TARGET_COUNT_TOTAL: break

            # [조치 3] 진행 정체 감지 (새 이미지가 없으면 바로 다음 태그)
            if collected_in_location == saved_before_scroll:
                print(" (정체)", end="")
                break

            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

        print(f" 완료 (누적: {collected_in_location}장)")

driver.quit()
print("\n🎉 모든 수집 공정이 완료되었습니다.")


📍 [구포시장] 수집 시작
  🔍 #구포시장 -> 📥 수집중... (정체) 완료 (누적: 18장)
  🔍 #구포시장뷰 -> 📥 수집중... (정체) 완료 (누적: 18장)
  🔍 #구포시장포토존 -> 📥 수집중... (정체) 완료 (누적: 18장)
  🔍 #구포시장출사 -> ⚠️ 로딩 실패/결과 없음 (Skip)
  🔍 #구포시장나들이 -> 📥 수집중... (정체) 완료 (누적: 36장)
  🔍 #구포시장여행 -> 📥 수집중... (정체) 완료 (누적: 36장)
  🔍 #구포시장야경 -> 📥 수집중... (정체) 완료 (누적: 37장)
  🔍 #구포시장광장 -> 📥 수집중... (정체) 완료 (누적: 37장)

📍 [국제시장] 수집 시작
  🔍 #국제시장 -> 📥 수집중... (정체) 완료 (누적: 0장)
  🔍 #국제시장뷰 -> 📥 수집중... (정체) 완료 (누적: 2장)
  🔍 #국제시장포토존 -> 📥 수집중... (정체) 완료 (누적: 5장)
  🔍 #국제시장출사 -> 📥 수집중... (정체) 완료 (누적: 5장)
  🔍 #국제시장나들이 -> 📥 수집중... (정체) 완료 (누적: 23장)
  🔍 #국제시장여행 -> 📥 수집중... (정체) 완료 (누적: 54장)
  🔍 #국제시장야경 -> 📥 수집중... (정체) 완료 (누적: 54장)
  🔍 #국제시장광장 -> 📥 수집중... (정체) 완료 (누적: 54장)

📍 [금강공원] 수집 시작
  🔍 #금강공원 -> 📥 수집중... (정체) 완료 (누적: 18장)
  🔍 #금강공원뷰 -> 📥 수집중... (정체) 완료 (누적: 19장)
  🔍 #금강공원포토존 -> 📥 수집중... (정체) 완료 (누적: 20장)
  🔍 #금강공원출사 -> 📥 수집중... (정체) 완료 (누적: 20장)
  🔍 #금강공원나들이 -> 📥 수집중... (정체) 완료 (누적: 20장)
  🔍 #금강공원여행 -> 📥 수집중... (정체) 완료 (누적: 20장)
  🔍 #금강공원야경 -> 📥 수집중... (정체) 완료 (누적: 20장

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time, os, requests, uuid
from datetime import datetime
from PIL import Image
from io import BytesIOid
import imagehash

# --- 1. 설정: 장소별 유사어(Aliases) 정의 ---
location_map = {
    "국제시장": ["국제시장", "부산국제시장", "gukjemarket"],
    "구포시장": ["구포시장", "구포개시장", "gupomarket"],
    "금강공원": ["금강공원", "부산금강공원", "geumgangpark"],
    "누리마루": ["누리마루", "누리마루apcc하우스", "nurimaru"],
    "더베이101": ["더베이101", "thebay101", "해운대더베이"],
    "동래온천": ["동래온천", "동래온천거리", "dongnaehotspring"],
    "동래읍성": ["동래읍성", "동래읍성지", "dongnaeeupseong"],
    "문화빙상센터": ["문화빙상센터", "북구문화빙상센터", "busanicerink"],
    "벡스코": ["벡스코", "BEXCO", "벡스코광장"],
    "보수동책방골목": ["보수동책방골목", "책방골목", "bosudongbookstreet"],
    "부전마켓타운": ["부전마켓타운", "부전시장", "bujeonmarket"],
    "부산어촌민속관": ["부산어촌민속관", "어촌민속관", "busanfishingvillage"],
    "비프광장": ["비프광장", "BIFF광장", "biffsquare"],
    "성지곡수원지": ["성지곡수원지", "어린이대공원수원지", "seongjigok"],
    "송상현광장": ["송상현광장", "송상현광장야경", "songsanghyeon"],
    "송정해수욕장": ["송정해수욕장", "송정바다", "songjeongbeach"],
    "부산 영화의전당": ["영화의전당", "부산시네마센터", "dureraum"],
    "용두산공원": ["용두산공원", "용두산타워", "yongdusampark"],
    "자갈치시장": ["자갈치시장", "부산자갈치시장", "jagalchimarket"],
    "장산": ["장산", "장산등산", "jangsanmountain"],
    "전포카페거리": ["전포카페거리", "전포사잇길", "jeonpocafe"],
    "화명생태공원": ["화명생태공원", "화명공원", "hwamyungpark"],
    "황령산": ["황령산", "황령산봉수대", "hwangnyeongsan"],
    "해운대블루라인파크": ["해운대블루라인파크", "블루라인파크", "bluelinepark"],
    "해운대해수욕장": ["해운대해수욕장", "해운대바다", "haeundaebeach"],
    "호천마을": ["호천마을", "남일바", "hocheonvillage"]
}

base_locations = sorted(location_map.keys())

# 첫 번째 키워드에만 적용할 변형 태그들
tag_variants = ["", "뷰", "포토존", "나들이", "여행", "야경", "광장", "핫플"]

EXCLUDE_ALT = ["텍스트", "포스터", "글자", "티켓", "안내문", "이벤트", "광고", "현수막"]

MAX_SCROLL_PER_TAG = 15   
TARGET_COUNT_TOTAL = 100  
STUCK_LIMIT = 2           

image_fingerprints = set()
visited_urls = set()

# --- 2. 브라우저 설정 ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.instagram.com/")
input("🚀 로그인을 완료하고 엔터를 치세요...")

# --- 3. 통합 수집 루프 ---
for main_loc in base_locations:
    save_path = f"image/{main_loc}"
    if not os.path.exists(save_path): os.makedirs(save_path)
    
    collected_in_location = 0
    print(f"\n📍 [{main_loc}] 통합 수집 시작 (목표: {TARGET_COUNT_TOTAL}장)")

    # [장소명 다양화 순회]
    search_keywords = location_map[main_loc]
    for idx, search_keyword in enumerate(search_keywords):
        if collected_in_location >= TARGET_COUNT_TOTAL: break
        
        # [핵심 로직] 첫 번째 검색어(idx=0)만 tag_variants 적용, 나머지는 기본 검색("")만 수행
        current_variants = tag_variants if idx == 0 else [""]
        
        for var in current_variants:
            if collected_in_location >= TARGET_COUNT_TOTAL: break
            
            target_tag = f"{search_keyword}{var}"
            print(f"  🔍 #{target_tag}", end=" ", flush=True)
            driver.get(f"https://www.instagram.com/explore/tags/{target_tag}/")

            try:
                # 결과가 나타날 때까지 대기 (최대 10초)
                wait = WebDriverWait(driver, 10)
                wait.until(EC.presence_of_element_located((By.TAG_NAME, "img")))
            except:
                print("-> Skip (No Content)")
                continue

            stuck_count = 0
            for scroll in range(MAX_SCROLL_PER_TAG):
                time.sleep(2.5) 
                img_elements = driver.find_elements(By.TAG_NAME, "img")
                saved_before_scroll = collected_in_location
                
                for img in img_elements:
                    try:
                        img_url = img.get_attribute("src")
                        alt_text = img.get_attribute("alt") or ""
                        
                        if not img_url or img_url in visited_urls or "static" in img_url: continue
                        if any(ex in alt_text for ex in EXCLUDE_ALT): continue

                        res = requests.get(img_url, timeout=5)
                        img_obj = Image.open(BytesIO(res.content))
                        
                        curr_hash = str(imagehash.average_hash(img_obj))
                        if curr_hash in image_fingerprints:
                            visited_urls.add(img_url)
                            continue
                        
                        if img_obj.size[0] < 400 or img_obj.size[1] < 400: continue

                        image_fingerprints.add(curr_hash)
                        visited_urls.add(img_url)
                        filename = f"{main_loc}_{uuid.uuid4().hex[:6]}.jpg"
                        img_obj.convert("RGB").save(os.path.join(save_path, filename), "JPEG")

                        collected_in_location += 1
                        if collected_in_location >= TARGET_COUNT_TOTAL: break
                    except: continue
                
                if collected_in_location >= TARGET_COUNT_TOTAL: break

                # 정체 감지 및 스크롤
                if collected_in_location == saved_before_scroll:
                    stuck_count += 1
                    if stuck_count >= STUCK_LIMIT: break
                else:
                    stuck_count = 0

                driver.execute_script("window.scrollBy(0, 1200);")

            print(f" -> 현재 누적: {collected_in_location}장")

driver.quit()
print("\n🎉 모든 데이터 통합 수집이 완료되었습니다.")


📍 [금강공원] 통합 수집 시작 (목표: 100장)
  🔍 #금강공원  -> 현재 누적: 28장
  🔍 #금강공원뷰  -> 현재 누적: 29장
  🔍 #금강공원포토존  -> 현재 누적: 30장
  🔍 #금강공원나들이  -> 현재 누적: 41장
  🔍 #금강공원여행  -> 현재 누적: 41장
  🔍 #금강공원야경  -> 현재 누적: 41장
  🔍 #금강공원광장  -> 현재 누적: 41장
  🔍 #금강공원핫플  -> 현재 누적: 41장
  🔍 #부산금강공원  -> 현재 누적: 78장
  🔍 #geumgangpark  -> 현재 누적: 100장

📍 [누리마루] 통합 수집 시작 (목표: 100장)
  🔍 #누리마루  -> 현재 누적: 25장
  🔍 #누리마루뷰  -> 현재 누적: 26장
  🔍 #누리마루포토존  -> 현재 누적: 27장
  🔍 #누리마루나들이  -> 현재 누적: 27장
  🔍 #누리마루여행  -> 현재 누적: 27장
  🔍 #누리마루야경  -> 현재 누적: 78장
  🔍 #누리마루광장  -> 현재 누적: 78장
  🔍 #누리마루핫플  -> 현재 누적: 78장
  🔍 #누리마루apcc하우스  -> 현재 누적: 78장
  🔍 #nurimaru  -> 현재 누적: 100장

📍 [더베이101] 통합 수집 시작 (목표: 100장)
  🔍 #더베이101  -> 현재 누적: 37장
  🔍 #더베이101뷰  -> 현재 누적: 64장
  🔍 #더베이101포토존  -> 현재 누적: 100장

📍 [동래온천] 통합 수집 시작 (목표: 100장)
  🔍 #동래온천  -> 현재 누적: 29장
  🔍 #동래온천뷰  -> 현재 누적: 29장
  🔍 #동래온천포토존  -> 현재 누적: 29장
  🔍 #동래온천나들이  -> 현재 누적: 29장
  🔍 #동래온천여행  -> 현재 누적: 29장
  🔍 #동래온천야경  -> 현재 누적: 29장
  🔍 #동래온천광장  -> 현재 누적: 29장
  🔍 #동래온천핫플  -> 현재 누적: 29장
  🔍 #동래온천거리  -> 현재 누적: 2

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time, os, requests, uuid
from datetime import datetime
from PIL import Image
from io import BytesIO
import imagehash

# --- 1. 설정: 장소별 맛집 키워드 설정 ---
# 딕셔너리의 첫 번째 값만 사용할 것이므로 대표 명칭을 앞에 둡니다.
location_map = {
    # "국제시장": ["국제시장맛집"],
    # "구포시장": ["구포시장맛집"],
    # "금강공원": ["금강공원맛집"],
    # "누리마루": ["해운대맛집"], # 누리마루 근처는 해운대로 확장
    # "더베이101": ["더베이101맛집"],
    # "동래온천": ["동래온천맛집"],
    # "동래읍성": ["동래읍성맛집"],
    # "문화빙상센터": ["덕천맛집"], # 문화빙상센터 인근 중심가
    # "벡스코": ["벡스코맛집"],
    # "보수동책방골목": ["보수동맛집"],
    # "부전마켓타운": ["부전시장맛집"],
    # "부산어촌민속관": ["화명동맛집"],
    # "비프광장": ["남포동맛집"],
    # "성지곡수원지": ["초읍맛집"],
    # "송상현광장": ["양정맛집"],
    # "송정해수욕장": ["송정맛집"],
    # "영화의전당": ["센텀맛집"],
    # "용두산공원": ["광복동맛집"],
    # "자갈치시장": ["자갈치시장맛집"],
    # "장산": ["장산역맛집"],
    # # "전포카페거리": ["전포카페거리맛집"],
    # "화명생태공원": ["화명생태공원맛집"],
    # "황령산": ["황령산맛집"],
    # "해운대블루라인파크": ["해운대블루라인파크맛집"],
    # "해운대해수욕장": ["해운대해수욕장맛집"],
    # "호천마을": ["호천마을맛집"]
}

base_locations = sorted(location_map.keys())

# 맛집 수집에 최적화된 태그 바리에이션 (음식 중심)
# 이번에는 요청하신 대로 "첫 번째 단어 + 맛집" 조합 위주로 구성됩니다.
tag_variants = [""]

# 홍보성 텍스트 이미지 제외 키워드 강화
EXCLUDE_ALT = ["텍스트", "포스터", "글자", "티켓", "안내문", "이벤트", "광고", "전단지", "메뉴판"]

MAX_SCROLL_PER_TAG = 15   
TARGET_COUNT_TOTAL = 30 # 폴더당 목표 수량
STUCK_LIMIT = 2           

image_fingerprints = set()
visited_urls = set()

# --- 2. 브라우저 설정 ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.instagram.com/")
input("🚀 로그인을 완료하고 엔터를 치세요...")

# --- 3. 맛집 통합 수집 루프 ---
for main_loc in base_locations:
    # 폴더명을 "관광지명맛집"으로 설정
    folder_name = f"{main_loc}맛집"
    save_path = f"image/{folder_name}"
    if not os.path.exists(save_path): os.makedirs(save_path)
    
    collected_in_location = 0
    print(f"\n📍 [{folder_name}] 수집 시작")

    # [수정] 딕셔너리의 가장 첫 번째 Value만 사용
    primary_keyword = location_map[main_loc][0]
    
    for var in tag_variants:
        if collected_in_location >= TARGET_COUNT_TOTAL: break
        
        target_tag = f"{primary_keyword}{var}"
        print(f"  🔍 #{target_tag}", end=" ", flush=True)
        driver.get(f"https://www.instagram.com/explore/tags/{target_tag}/")

        try:
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "img")))
        except:
            print("-> Skip")
            continue

        stuck_count = 0
        for scroll in range(MAX_SCROLL_PER_TAG):
            time.sleep(2.5) 
            img_elements = driver.find_elements(By.TAG_NAME, "img")
            saved_before_scroll = collected_in_location
            
            for img in img_elements:
                try:
                    img_url = img.get_attribute("src")
                    alt_text = img.get_attribute("alt") or ""
                    
                    if not img_url or img_url in visited_urls or "static" in img_url: continue
                    if any(ex in alt_text for ex in EXCLUDE_ALT): continue

                    res = requests.get(img_url, timeout=5)
                    img_obj = Image.open(BytesIO(res.content))
                    
                    curr_hash = str(imagehash.average_hash(img_obj))
                    if curr_hash in image_fingerprints:
                        visited_urls.add(img_url)
                        continue
                    
                    if img_obj.size[0] < 400 or img_obj.size[1] < 400: continue

                    image_fingerprints.add(curr_hash)
                    visited_urls.add(img_url)
                    
                    # 파일명: 관광지명맛집_고유ID
                    filename = f"{folder_name}_{uuid.uuid4().hex[:6]}.jpg"
                    img_obj.convert("RGB").save(os.path.join(save_path, filename), "JPEG")

                    collected_in_location += 1
                    if collected_in_location >= TARGET_COUNT_TOTAL: break
                except: continue
            
            if collected_in_location >= TARGET_COUNT_TOTAL: break

            if collected_in_location == saved_before_scroll:
                stuck_count += 1
                if stuck_count >= STUCK_LIMIT: break
            else:
                stuck_count = 0

            driver.execute_script("window.scrollBy(0, 1200);")

        print(f" -> 현재 누적: {collected_in_location}장")

driver.quit()
print("\n🎉 모든 맛집 데이터 수집이 완료되었습니다.")


📍 [해운대블루라인파크맛집] 수집 시작
  🔍 #해운대블루라인파크맛집  -> 현재 누적: 30장

📍 [해운대해수욕장맛집] 수집 시작
  🔍 #해운대해수욕장맛집  -> 현재 누적: 23장

📍 [호천마을맛집] 수집 시작
  🔍 #호천마을맛집  -> 현재 누적: 1장

📍 [화명생태공원맛집] 수집 시작
  🔍 #화명생태공원맛집  -> 현재 누적: 23장

📍 [황령산맛집] 수집 시작
  🔍 #황령산맛집  -> 현재 누적: 30장

🎉 모든 맛집 데이터 수집이 완료되었습니다.
